In [ ]:
```python
# %% [markdown]
# # Visualization of BSE Experiment Results
# This notebook visualizes results from 90 trials (3 noise levels × 2 agent mixes × 15 trials) and 15 baseline trials.
# Metrics: Profit, Volatility of Returns, Trade Frequency, Bid-Ask Spread.
# Results are stored in base_output/ (baseline) and output/mix_X/noise_Y/ (experiments).

# %% [code]
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from pathlib import Path

# Configuration
noise_levels = [0.0, 0.1, 0.2]
agent_mixes = ['mix_1', 'mix_2']  # 1+1, 1+1+1
trials_per_condition = 15
baseline_trials = 15
initial_balance = 10000  # Initial balance for proptraders
plt.style.use('seaborn')

# %% [markdown]
# ## 1. Compute Metrics

# %% [code]
def compute_metrics(trial_id, noise, mix=None):
    """Compute metrics for a single trial."""
    # Determine file path based on trial type
    if mix == 'baseline':
        output_dir = 'base_output'
    else:
        output_dir = f'output/{mix}/noise_{noise}'
    
    # File paths
    avg_bal_file = f'{output_dir}/{trial_id}_avg_balance.csv'
    blotter_file = f'{output_dir}/{trial_id}_blotters.csv'
    lob_file = f'{output_dir}/{trial_id}_LOB_frames.csv'
    tape_file = f'{output_dir}/{trial_id}_tape.csv'
    
    # Profit (from avg_balance.csv)
    avg_bal = pd.read_csv(avg_bal_file)
    profits = {}
    for agent in ['TrendFollower', 'MeanReverter', 'RLAgent', 'ZIC']:
        col = next((c for c in avg_bal.columns if c.startswith(agent)), None)
        if col:
            profit = avg_bal[col].iloc[-1] - initial_balance
            profits[agent] = profit
    
    # Volatility of Returns (from tape.csv)
    tape = pd.read_csv(tape_file)
    volatility = tape['Price'].std() if not tape.empty else 0
    
    # Trade Frequency (from blotters.csv)
    blotter = pd.read_csv(blotter_file)
    trade_counts = {}
    for agent in ['TrendFollower', 'MeanReverter', 'RLAgent', 'ZIC']:
        count = len(blotter[blotter['tid'].str.startswith(agent[0])])
        trade_counts[agent] = count
    
    # Bid-Ask Spread (from LOB_frames.csv)
    lob = dd.read_csv(lob_file, usecols=['best_bid', 'best_ask'])
    lob = lob.compute()  # Convert to pandas
    spread = (lob['best_ask'] - lob['best_bid']).mean() if not lob.empty else 0
    
    return {
        'trial_id': trial_id,
        'noise': noise,
        'mix': mix or 'baseline',
        'profits': profits,
        'volatility': volatility,
        'trade_counts': trade_counts,
        'spread': spread
    }

# Collect metrics for all trials
metrics = []
for noise in noise_levels:
    for mix in agent_mixes:
        for t in range(1, trials_per_condition + 1):
            trial_idx = t + (int(noise * 30) + (1 if mix == 'mix_2' else 0) * 15)
            trial_id = f'exp_d001_i10_{trial_idx:04d}'
            metrics.append(compute_metrics(trial_id, noise, mix))
for t in range(baseline_trials):
    trial_id = f'baseline_zic_{t:04d}'
    metrics.append(compute_metrics(trial_id, 'baseline', 'baseline'))

# Convert to DataFrame
metrics_df = pd.DataFrame([
    {
        'trial_id': m['trial_id'],
        'noise': m['noise'],
        'mix': m['mix'],
        'agent': agent,
        'profit': m['profits'].get(agent, 0),
        'trades': m['trade_counts'].get(agent, 0),
        'volatility': m['volatility'],
        'spread': m['spread']
    }
    for m in metrics for agent in ['TrendFollower', 'MeanReverter', 'RLAgent', 'ZIC']
    if m['profits'].get(agent, 0) != 0 or m['trade_counts'].get(agent, 0) != 0
])

# %% [markdown]
# ## 2. Visualize Results

# %% [code]
# Profit Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x='noise', y='profit', hue='agent', data=metrics_df)
plt.title('Agent Profits by Noise Level and Mix')
plt.xlabel('Noise Level')
plt.ylabel('Profit')
plt.legend(title='Agent', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('profit_boxplot.png', bbox_inches='tight')
plt.close()

# Volatility Lineplot
volatility_df = metrics_df.groupby(['noise', 'mix'])['volatility'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(x='noise', y='volatility', hue='mix', data=volatility_df)
plt.title('Price Volatility by Noise Level')
plt.xlabel('Noise Level')
plt.ylabel('Volatility (Std Dev of Prices)')
plt.tight_layout()
plt.savefig('volatility_lineplot.png')
plt.close()

# Trade Frequency Barplot
trade_df = metrics_df.groupby(['noise', 'mix', 'agent'])['trades'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.catplot(x='noise', y='trades', hue='agent', col='mix', kind='bar', data=trade_df, height=5, aspect=1.2)
plt.suptitle('Trade Frequency by Noise Level and Agent Mix', y=1.05)
plt.tight_layout()
plt.savefig('trade_frequency_barplot.png')
plt.close()

# Bid-Ask Spread Violinplot
plt.figure(figsize=(10, 6))
sns.violinplot(x='noise', y='spread', hue='mix', data=metrics_df)
plt.title('Bid-Ask Spread by Noise Level')
plt.xlabel('Noise Level')
plt.ylabel('Average Bid-Ask Spread')
plt.tight_layout()
plt.savefig('spread_violinplot.png')
plt.close()

# %% [markdown]
# ## 3. Save Summary Statistics
metrics_df.groupby(['noise', 'mix', 'agent']).agg({
    'profit': ['mean', 'std'],
    'trades': ['mean', 'std'],
    'volatility': ['mean'],
    'spread': ['mean']
}).to_csv('summary_stats.csv')

# %% [code]
print("Visualization complete. Plots saved: profit_boxplot.png, volatility_lineplot.png, trade_frequency_barplot.png, spread_violinplot.png")
print("Summary statistics saved to summary_stats.csv")
```